# Gets list of candidate reactions for overexpression
## Written by: Wheaton Schroeder
### Latest version: 08/30/2023

#### make necessary imports

In [7]:
import pandas as pd
import cobra
import re
import shutil

#### Set run parameters

In [8]:
harvest = "H1"
drought_file = "./iPotri2999D.xml"

#### read in the FVA results

In [9]:
#read in the drought FVA result
drought_df = pd.read_table('./GAMS/FVA_result_'+harvest+'.txt')

#get list of reactions in the drought
drought_rxn_list = drought_df.rxn.to_list()

#clean up by removing the unnecessary spaces
for rxn in drought_rxn_list:

  #cleaned reaction
  cleaned_rxn = rxn.replace(" ","")

  drought_df.loc[drought_rxn_list.index(rxn),'rxn'] = cleaned_rxn
  drought_rxn_list[drought_rxn_list.index(rxn)] = cleaned_rxn

#set the reaction as the index
drought_df = drought_df.set_index('rxn')

#for debugging
drought_df

,LB,min,max,UB,ModelStat,SolveStat
rxn,,,,,,
RXN_UDPREDUCT_RXN_p__L___LIGHT,-10000.0,0.000000,0.000000,0.0,1.0,1.0
RXN_E2C_mg2__L___LIGHT,-10000.0,0.000000,0.000000,10000.0,1.0,1.0
RXN_ICOSANOYL_COA_SYNTHASE_RXN_er__L___LIGHT,-10000.0,0.000000,0.000000,10000.0,1.0,1.0
RXN_GPH_RXN_er__L___LIGHT,0.0,0.000000,11.335471,10000.0,1.0,1.0
RXN_RXN_15816__L___LIGHT,-10000.0,0.000000,120.744914,10000.0,1.0,1.0
...,...,...,...,...,...,...
SK_MET_carbon_dioxide_c__S___LIGHT,-1000.0,-1.069132,-1.069132,0.0,1.0,1.0
DM_MET_starch_p__L___LIGHT,0.0,0.654897,0.654897,1000.0,1.0,1.0
DM_MET_starch_p__S___LIGHT,0.0,0.328273,0.328273,1000.0,1.0,1.0


In [10]:
cont_df = pd.read_table('./GAMS/FVA_result_C.txt')

#get list of reactions in the control
cont_rxn_list = cont_df.rxn.to_list()

#clean up by removing the unnecessary spaces
for rxn in cont_rxn_list:

  #cleaned reaction
  cleaned_rxn = rxn.replace(" ","")

  cont_df.loc[cont_rxn_list.index(rxn),'rxn'] = cleaned_rxn
  cont_rxn_list[cont_rxn_list.index(rxn)] = cleaned_rxn

#set the reaction as the index
cont_df = cont_df.set_index('rxn')

#for debugging
cont_df

,LB,min,max,UB,ModelStat,SolveStat
rxn,,,,,,
RXN_UDPREDUCT_RXN_p__L___LIGHT,-10000.0,0.000000,0.000000,0.0,1.0,1.0
RXN_E2C_mg2__L___LIGHT,-10000.0,0.000000,0.000000,10000.0,1.0,1.0
RXN_RXN_11755_er__L___LIGHT,0.0,0.000000,178.893509,10000.0,1.0,1.0
RXN_ICOSANOYL_COA_SYNTHASE_RXN_er__L___LIGHT,-10000.0,0.000000,0.000000,10000.0,1.0,1.0
RXN_GPH_RXN_er__L___LIGHT,0.0,0.000000,18.181554,10000.0,1.0,1.0
...,...,...,...,...,...,...
SK_MET_carbon_dioxide_c__S___LIGHT,-1000.0,-1.141754,-1.141753,0.0,1.0,1.0
DM_MET_starch_p__L___LIGHT,0.0,0.703488,0.703488,1000.0,1.0,1.0
DM_MET_starch_p__S___LIGHT,0.0,0.354921,0.354921,1000.0,1.0,1.0


#### Read in reactions that are downregulated in the leaf

##### leaf

In [11]:
#read the list of genes downregulated in the leaf
down_leaf_f = open("./DGE_analysis/downRegulated_LEAF.txt")

#initialize a list for them
down_leaf = list()

#read the file into the list
for gene in down_leaf_f:

  #remove newline characters
  gene_temp = gene.replace("\n","")

  #remove unnecessary spaces
  gene_temp = gene_temp.replace(" ","")
  
  #add to the 
  down_leaf.append(gene_temp)

##### root

In [12]:
#read the list of genes downregulated in the root
down_root_f = open("./DGE_analysis/downRegulated_ROOT.txt")

#initialize a list for them
down_root = list()

#read the file into the list
for gene in down_root_f:

  #remove newline characters
  gene_temp = gene.replace("\n","")

  #remove unnecessary spaces
  gene_temp = gene_temp.replace(" ","")
  
  #add to the 
  down_root.append(gene_temp)

##### stem

In [13]:
#read the list of genes downregulated in the root
down_stem_f = open("./DGE_analysis/downRegulated_STEM.txt")

#initialize a list for them
down_stem = list()

#read the file into the list
for gene in down_stem_f:

  #remove newline characters
  gene_temp = gene.replace("\n","")

  #remove unnecessary spaces
  gene_temp = gene_temp.replace(" ","")
  
  #add to the 
  down_stem.append(gene_temp)

#### Read in upregulated gene sets

##### Leaf

In [14]:
#read the list of genes downregulated in the leaf
up_leaf_f = open("./DGE_analysis/upregulated_LEAF.txt")

#initialize a list for them
up_leaf = list()

#read the file into the list
for gene in up_leaf_f:

  #remove newline characters
  gene_temp = gene.replace("\n","")

  #remove unnecessary spaces
  gene_temp = gene_temp.replace(" ","")
  
  #add to the 
  up_leaf.append(gene_temp)

##### Root

In [15]:
#read the list of genes downregulated in the root
up_root_f = open("./DGE_analysis/upregulated_ROOT.txt")

#initialize a list for them
up_root = list()

#read the file into the list
for gene in up_root_f:

  #remove newline characters
  gene_temp = gene.replace("\n","")

  #remove unnecessary spaces
  gene_temp = gene_temp.replace(" ","")
  
  #add to the 
  up_root.append(gene_temp)

##### Stem

In [16]:
#read the list of genes downregulated in the root
up_stem_f = open("./DGE_analysis/upregulated_STEM.txt")

#initialize a list for them
up_stem = list()

#read the file into the list
for gene in up_stem_f:

  #remove newline characters
  gene_temp = gene.replace("\n","")

  #remove unnecessary spaces
  gene_temp = gene_temp.replace(" ","")
  
  #add to the 
  up_stem.append(gene_temp)

#### Read in the drought model for GPR

In [17]:
drought_model = cobra.io.read_sbml_model(drought_file)

print("success")

No objective coefficients in model. Unclear what should be optimized


success


#### identify candidates, report on results simultaneously

In [18]:
#open file to write candidates to
kd_cand_f = open("./kd_results/kd_candidates_"+harvest+".txt","w")

kd_cand_f.write("/")

#since we only want reactions in drought and control, iterating over a single one is OK
for rxn in cont_rxn_list:
    
    #check if in drought, do nothing it it is not
    if rxn in drought_rxn_list:

      print("\nanalyzing reaction ",rxn)
        
      #at this point, we know the reaction is in both drought and control
      #get min and max values in control and drought
      fva_min_d = drought_df.loc[rxn,'min']
      fva_max_d = drought_df.loc[rxn,'max']
      fva_min_c = cont_df.loc[rxn,'min']
      fva_max_c = cont_df.loc[rxn,'max']

      print("min_d: ",str(fva_min_d),"\tmax_d: ",str(fva_max_d),"min_c: ",str(fva_min_c),"\tmax_c: ",str(fva_max_c))

      #get the maximum absolute values for drought and control
      max_abs_d = max(abs(fva_min_d),abs(fva_max_d))
      max_abs_c = max(abs(fva_min_c),abs(fva_max_c))

      #do math to make the comparison
      comparison = max_abs_d - max_abs_c

      #ensure comparison is significant in tolerance
      if(comparison >= 1E-7):

        #for debugging
        print("increased flux space magnitude in drought: ",rxn,"\tdrought max abs: ",max_abs_d,"\tcontrol max abs: ",max_abs_c)

        #determine if it is downregulated in the DGE analysis
        downregulated = False

        #next, check GPR, see if downregulated in the tissue where the change in flux is present
        match = re.search(r'__(\w)___(LIGHT|DARK)', rxn)

        try:

          tissue = match.group(1)

          #get set of associated genes
          genes = drought_model.reactions[drought_model.reactions.index(rxn)].genes

          #get reaction tissue

          #for each gene
          for gene in genes:

            print(gene.id)

            #from the tissue, search if that is downregulated in that tissue
            if tissue == "S":

              if gene.id in down_stem:

                print("gene "+gene.id+" is downregulated in the stem!")

                downregulated = True

              elif gene.id in up_stem:

                print("gene "+gene.id+" is upregulated in the stem!")

              else:

                print("gene "+gene.id+" is not siginficantly differently regulated in the stem")

            elif tissue == "R":

              if gene.id in down_root:

                print("gene "+gene.id+" is downregulated in the root!")

                downregulated = True

              elif gene.id in up_stem:

                print("gene "+gene.id+" is upregulated in the root!")

              else:

                print("gene "+gene.id+" is not siginficantly differently regulated in the root")

            elif tissue == "L":

              if gene.id in down_root:

                print("gene "+gene.id+" is downregulated in the leaf!")

                downregulated = True

              elif gene.id in up_stem:

                print("gene "+gene.id+" is upregulated in the leaf!")

              else:

                print("gene "+gene.id+" is not siginficantly differently regulated in the leaf")

          #if here then the reaction a cadidate
          if downregulated:

            kd_cand_f.write("\n\""+rxn+"\"")

        except:

          print("need to analyze ",rxn," manually!!!")

kd_cand_f.write("\n/")

kd_cand_f.close()

#copy the file into the gams folder
shutil.copyfile("./kd_results/kd_candidates_"+harvest+".txt","./GAMS/kd_candidates_"+harvest+".txt")


analyzing reaction  RXN_UDPREDUCT_RXN_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_E2C_mg2__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_ICOSANOYL_COA_SYNTHASE_RXN_er__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_GPH_RXN_er__L___LIGHT
min_d:  0.0 	max_d:  11.33547149 min_c:  0.0 	max_c:  18.18155443

analyzing reaction  RXN_RXN_15816__L___LIGHT
min_d:  0.0 	max_d:  120.74491395 min_c:  0.0 	max_c:  130.67464335

analyzing reaction  RXN_Trans_oxygen_molecule_m__L___LIGHT
min_d:  -4498.8923499 	max_d:  60.37245697 min_c:  -4499.50567833 	max_c:  65.33732167

analyzing reaction  RXN_Trans_cpd_12860_er__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_GLUTATHIONE_REDUCT_NADPH_RXN_c__L___LIGHT
min_d:  -60.37245697 	max_d:  0.0 min_c:  -65.33732167 	max_c:  0.0

analyzing reaction  RXN_RXN1F_155_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 m

min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_RXN_8042_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_ICOSANOYL_COA_SYNTHASE_RXN_g__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_3_6_3_44_RXN_c2er__L___LIGHT
min_d:  0.0 	max_d:  320.22706973 min_c:  0.0 	max_c:  357.78701755

analyzing reaction  RXN_RXN_8642_g__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_PHOSGLYPHOS_RXN_p__L___LIGHT
min_d:  62.10059466 	max_d:  382.32766439 min_c:  0.0 	max_c:  406.89711624

analyzing reaction  RXN_GLYOHMETRANS_RXN_m__L___LIGHT
min_d:  -120.74491395 	max_d:  0.0 min_c:  -130.67464335 	max_c:  0.0

analyzing reaction  RXN_3_6_3_44_RXN_c2g__L___LIGHT
min_d:  0.0 	max_d:  320.22706973 min_c:  0.0 	max_c:  357.78701755

analyzing reaction  RXN_3_6_3_44_RXN_c2x__L___LIGHT
min_d:  0.0 	max_d:  320.22706973 min_c:  0.0 	max_c:  357.78701755

analyzing reaction  R

analyzing reaction  RXN_THYMIDYLATESYN_RXN_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_Trans_super_oxide_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_RXN0_5199_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_DTDPKIN_RXN_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_ASPAMINOTRANS_RXN_g__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_RXN_1106_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_4_2_1_58_RXN_m__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_Trans_stearoyl_coa_g__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_Trans_udp_c2p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_MALATE_DEHYDROGENASE_NADP_RXN_m__L___LIGHT
min_d:  -10

analyzing reaction  RXN_KETOHEXOKINASE_RXN_m__L___LIGHT
min_d:  0.0 	max_d:  320.22706973 min_c:  0.0 	max_c:  357.78701755

analyzing reaction  RXN_2_1_1_103_RXN_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_1TRANSKETO_RXN_p__L___LIGHT
min_d:  11.29101721 	max_d:  38.79059906 min_c:  8.61580679 	max_c:  41.91271656

analyzing reaction  RXN_DXPREDISOM_RXN_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_CDPDIGLYSYN_RXN_m__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_DUTP_PYROP_RXN_p__L___LIGHT
min_d:  0.0 	max_d:  320.22706973 min_c:  0.0 	max_c:  357.78701755

analyzing reaction  RXN_RXN_10772_m__L___LIGHT
min_d:  0.0 	max_d:  17.79039276 min_c:  0.0 	max_c:  29.1221991

analyzing reaction  RXN_Trans_datp_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_Trans_epoxysqualene_er__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max

min_d:  0.0 	max_d:  4.00283837 min_c:  0.0 	max_c:  6.26127281

analyzing reaction  RXN_4_COUMARATE__COA_LIGASE_RXN_c__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_AMP_DEPHOSPHORYLATION_RXN_g__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_SUPEROX_DISMUT_RXN_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_RXN_10763_g__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_Trans_hydrogen_peroxide_p__L___LIGHT
min_d:  -120.74491395 	max_d:  120.74491395 min_c:  -130.67464335 	max_c:  130.67464335

analyzing reaction  RXN_RXN_14014_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_Trans_arg_p__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_Trans_trehalose_6p_m__L___LIGHT
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_3_HYDROXBUTYRYL_COA_DEH

...

min_d:  -0.00043648 	max_d:  -0.00043648 min_c:  -0.00045467 	max_c:  -0.00045467

analyzing reaction  RXN_RXN_9657_m__R___DARK
min_d:  -0.00198424 	max_d:  -0.00198424 min_c:  -0.00206691 	max_c:  -0.00206691

analyzing reaction  RXN_Trans_oxygen_molecule_c2er__R___DARK
min_d:  0.00415004 	max_d:  0.00416762 min_c:  0.00451407 	max_c:  0.00452949

analyzing reaction  RXN_RXN_7737_m__R___DARK
min_d:  -0.00021689 	max_d:  -0.00021689 min_c:  -0.00022593 	max_c:  -0.00022593

analyzing reaction  RXN_Trans_ribuloses_g__R___DARK
min_d:  -0.00050567 	max_d:  -0.00049449 min_c:  -0.00047721 	max_c:  -0.00046763
increased flux space magnitude in drought:  RXN_Trans_ribuloses_g__R___DARK 	drought max abs:  0.00050567 	control max abs:  0.00047721

analyzing reaction  RXN_RXN_10770_p__R___DARK
min_d:  -10000.0 	max_d:  -0.05281595 min_c:  -10000.0 	max_c:  -0.05194416

analyzing reaction  RXN_DUDPKIN_RXN_p__R___DARK
min_d:  1.702e-05 	max_d:  1.739e-05 min_c:  1.774e-05 	max_c:  1.811e-05

anal

min_d:  0.0 	max_d:  1.05e-06 min_c:  0.0 	max_c:  1.17e-06

analyzing reaction  RXN_RXN_5642_p__R___DARK
min_d:  0.00013832 	max_d:  0.0001391 min_c:  0.00014359 	max_c:  0.00014489

analyzing reaction  RXN_PRPPSYN_RXN_m__R___DARK
min_d:  0.0002172 	max_d:  0.00021945 min_c:  0.00022514 	max_c:  0.0002286

analyzing reaction  RXN_RXN_8309_p__R___DARK
min_d:  1.606e-05 	max_d:  1.606e-05 min_c:  1.673e-05 	max_c:  1.673e-05

analyzing reaction  RXN_E2C_oxygen_molecule__R___DARK
min_d:  5.58560324 	max_d:  5.58560421 min_c:  5.89125349 	max_c:  5.89126983

analyzing reaction  RXN_RXN_12420_p__R___DARK
min_d:  0.11910348 	max_d:  0.11938917 min_c:  0.12558553 	max_c:  0.12588353

analyzing reaction  RXN_RXN66_310_er__R___DARK
min_d:  0.00026125 	max_d:  0.00026125 min_c:  0.00027214 	max_c:  0.00027214

analyzing reaction  RXN_RXN_10625_p__R___DARK
min_d:  -2.032e-05 	max_d:  -2.032e-05 min_c:  -2.116e-05 	max_c:  -2.116e-05

analyzing reaction  RXN_FPPSYN_RXN_p__R___DARK
min_d:  6.84e-0

min_d:  0.0 	max_d:  5.2e-07 min_c:  0.0 	max_c:  5.8e-07

analyzing reaction  RXN_GLUC1PURIDYLTRANS_RXN_g__R___DARK
min_d:  -1.851e-05 	max_d:  -1.498e-05 min_c:  -2.084e-05 	max_c:  -1.552e-05

analyzing reaction  RXN_Trans_uridine_er__R___DARK
min_d:  -5.2e-07 	max_d:  0.0 min_c:  -5.8e-07 	max_c:  0.0

analyzing reaction  RXN_1_1_1_34_RXN_er__R___DARK
min_d:  -7e-07 	max_d:  0.0 min_c:  -7.1e-07 	max_c:  0.0

analyzing reaction  RXN_RXN_16393_m__R___DARK
min_d:  0.0 	max_d:  1.19e-06 min_c:  0.0 	max_c:  2.8e-06

analyzing reaction  RXN_UTPHEXPURIDYLYLTRANS_RXN_m__R___DARK
min_d:  0.0 	max_d:  5.6e-07 min_c:  0.0 	max_c:  8.2e-07

analyzing reaction  RXN_RXN_9661_m__R___DARK
min_d:  -0.00198424 	max_d:  -0.00198424 min_c:  -0.00206691 	max_c:  -0.00206691

analyzing reaction  RXN_RXN0_5199_g__R___DARK
min_d:  -1.05e-06 	max_d:  0.0 min_c:  -1.17e-06 	max_c:  0.0

analyzing reaction  RXN_RXN_10697_p__R___DARK
min_d:  -0.0 	max_d:  0.0 min_c:  -0.0 	max_c:  0.0

analyzing reaction  R

min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_Triose_phosphate_antiporter_gap__S___DARK
min_d:  0.0 	max_d:  5.85304088 min_c:  0.0 	max_c:  4.10121023
increased flux space magnitude in drought:  RXN_Triose_phosphate_antiporter_gap__S___DARK 	drought max abs:  5.85304088 	control max abs:  4.10121023

analyzing reaction  RXN_1_18_1_2_RXN__S___DARK
min_d:  0.0 	max_d:  2500.00008336 min_c:  0.0 	max_c:  2500.09001746

analyzing reaction  RXN_E2S_CO2_DARK
min_d:  10.69131832 	max_d:  10.6913201 min_c:  11.41753494 	max_c:  11.41753692

analyzing reaction  RXN_SUCROSE_PHOSPHATASE_RXN_c__S___DARK
min_d:  0.0 	max_d:  0.0 min_c:  0.0 	max_c:  0.0

analyzing reaction  RXN_3_2_1_1_RXN_p__L___DARK
min_d:  0.65489653 	max_d:  0.6548969 min_c:  0.70348769 	max_c:  0.7034881

analyzing reaction  RXN_3_2_1_1_RXN_c__L___DARK
min_d:  0.0 	max_d:  5.2e-07 min_c:  0.0 	max_c:  5.8e-07

analyzing reaction  RXN_3_2_1_1_RXN_p__S___DARK
min_d:  0.32888469 	max_d:  0.3288857 mi

'./GAMS/kd_candidates_H1.txt'